In [ ]:
import pandas as pd
import numpy as np
import spacy
import re
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import drive
drive.mount('/content/drive')
import warnings; warnings.simplefilter('ignore')

Mounted at /content/drive


In [ ]:
# global variables

# path to dataset
path = '/content/drive/MyDrive/books.csv'


# create empty dictionary to store predicted data types
predicted_data_types = {}

# type map for nlp predictions
type_map = {
            'org': ['name', 'brand', 'company', 'organization', 'firm', 'hotel', 'industry'],
            'person': ['name', 'relation', 'first name', 'last name', 'middle name'],
            'gpe': ['place', 'destination', 'origin', 'landmark', 'address', 'country', 'city', 'state'],
            'fac': ['building', 'airport', 'highway', 'bridge', 'station', 'metro', 'vehicle', 'hospital'],
            'norp': ['nationality', 'religion', 'political party', 'caste', 'creed', 'race', 'group', 'congregation'],
            'loc': ['location', 'place', 'mountain range', 'water body', 'coordinates'],
            'product': ['item', 'commodity', 'product', 'machine', 'technology', 'service', 'transportation', 'book', 'investment', 'insurance', 'description'],
            'event': ['sport', 'war', 'hurricane', 'tornado', 'attack', 'tsunami', 'calamity'],
            'work_of_art': ['movie', 'book', 'music', 'song', 'art', 'pop', 'culture', 'band'],
            'time': ['arrival', 'departure', 'time zone', 'standard time', 'expiration date', 'manufacture date', 'last seen', 'occurance', 'timing'],
            'date': ['ID','serial number','identification number', 'birthday', 'anniversary', 'expiration date', 'manufacture date'],
            'percent': ['percentage', 'ratio', 'range', 'factor', 'length', 'breadth', 'height', 'altitude', 'mean sea level', 'mean', 'median', 'sensex', 'statistics', 'nifty', 'rating', 'cross-section', 'dose', 'weight', 'number', 'enumeration'],
            'money': ['unit', 'worth', 'value', 'price', 'cost', 'expense', 'currency', 'quantity', 'amount'],
            'amount': ['percentage', 'ratio', 'range', 'factor', 'length', 'breadth', 'height', 'altitude', 'mean sea level', 'mean', 'median', 'sensex', 'statistics', 'nifty', 'rating', 'cross-section', 'dose', 'weight', 'number', 'enumeration'],
            'currency': ['unit', 'worth', 'value', 'price', 'cost', 'expense', 'currency', 'quantity', 'amount', 'gain', 'profit', 'loss', 'incomme', 'salary', 'revenue', 'net worth', 'net profit', 'net loss', 'capitol', 'debit', 'credit'],
            'measurement': ['unit', 'measurement', 'length', 'breadth', 'height', 'width', 'cross-section', 'quantity', 'amount', 'number of', 'enumeration', 'debit', 'credit'],
            'cardinal': ['ID', 'identification number', 'number plate', 'latitiude', 'longitude', 'coordinates', 'paper id', 'vehicle number', 'code', 'zipcode', 'contact number', 'shipping details', 'passcode', 'password', 'identity', 'email id', 'username', 'medicine', 'url'],
            'ordinal' : ['rating', 'numerical indicator', 'size', 'level', 'status'],
            'flag' : ['Yes','No','Male','Female', '0','1','Y','N','F','M']
}



In [ ]:
def classify_column_datatype(df):    
    # iterate through each column in the dataframe
    for column in df.columns:
        # check data type of column
        dtype = str(df[column].dtype)

        # predict data type based on dtype
        if dtype == 'object':
            if df[column].str.match(r'^\d+$').all():
                predicted_data_types[column] = 'numerical integer'

            elif df[column].str.match(r'^\d+\.\d+$').all():
                predicted_data_types[column] = 'numerical float'

            elif df[column].str.match(r'\b\d{2}/\d{2}/\d{4}\b').all():
                predicted_data_types[column] = 'datetime'
            
            elif df[column].str.match(r'\b\d{1,2}-\d{2}-\d{4}\b').all() or pd.to_datetime(df[column], errors='coerce').notnull().all():
                predicted_data_types[column] = 'datetime'

            elif df[column].str.match(r'\b\d{4}-\d{2}-\d{2}\b').all() or pd.to_datetime(df[column], errors='coerce').notnull().all():
                predicted_data_types[column] = 'datetime'

            elif df[column].str. match(r'\b\d{4}/\d{2}/\d{2}\b').all() or pd.to_datetime(df[column], errors='coerce').notnull().all():
                predicted_data_types[column] = 'datetime'

            elif df[column].str.match(r'\b\d{1,2}:\d{2}\b').all() or pd.to_datetime(df[column], errors='coerce').notnull().all():
                predicted_data_types[column] = 'datetime'

            elif df[column].str.match(r'\b\d{1,2} (am|pm)\b').all() or pd.to_datetime(df[column], errors='coerce').notnull().all():
                predicted_data_types[column] = 'time'    
            else:
                predicted_data_types[column] = 'categorical'
        elif dtype == 'int64':
            if df[column].isnull().any():
                predicted_data_types[column] = 'float'
            else:
                predicted_data_types[column] = 'int'
        elif dtype == 'float64':
            if df[column].isnull().any():
                predicted_data_types[column] = 'float'
            elif df[column].apply(lambda x: x.is_integer()).all():
                predicted_data_types[column] = 'int'
            else:
                predicted_data_types[column] = 'float'
        elif dtype == 'bool':
            predicted_data_types[column] = 'bool'
        else:
            predicted_data_types[column] = 'unknown'

    return predicted_data_types
    # print predicted data types
    for column, data_type in predicted_data_types.items():
        print(f"{column}: {data_type}")

In [ ]:
def preprocess_df(path):
  
  read1 = pd.read_csv(path , on_bad_lines='skip')
  read1.shape
  read1.isnull()
  read1.isnull().sum()
  df_var=read1.fillna(" ")
  df_var.isnull().sum()
  predicted_data_types = classify_column_datatype(df_var)
  # df_var.head()
  return df_var

In [ ]:
df1 = preprocess_df('/content/drive/MyDrive/HPCC_Datasets/books.csv')
df1

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic
...,...,...,...,...,...,...,...,...,...,...,...,...
11118,45631,Expelled from Eden: A William T. Vollmann Reader,William T. Vollmann/Larry McCaffery/Michael He...,4.06,1560254416,9781560254416,eng,512,156,20,12/21/2004,Da Capo Press
11119,45633,You Bright and Risen Angels,William T. Vollmann,4.08,0140110879,9780140110876,eng,635,783,56,12/1/1988,Penguin Books
11120,45634,The Ice-Shirt (Seven Dreams #1),William T. Vollmann,3.96,0140131965,9780140131963,eng,415,820,95,8/1/1993,Penguin Books
11121,45639,Poor People,William T. Vollmann,3.72,0060878827,9780060878825,eng,434,769,139,2/27/2007,Ecco


In [ ]:

# print predicted data types
for column, data_type in predicted_data_types.items():
  print(f"{column}: {data_type}")

bookID: int
title: categorical
authors: categorical
average_rating: float
isbn: categorical
isbn13: int
language_code: categorical
  num_pages: int
ratings_count: int
text_reviews_count: int
publication_date: categorical
publisher: categorical


In [ ]:
#NER on csv fILE applies on every value in a column and returns the column
nlp = spacy.load("en_core_web_sm")

def get_entities(text):
    doc = nlp(text)
    return [(ent.label_) for ent in doc.ents]

# this function assigns ner value to new copy of df and returns the new df
# Extracts majority of each column and stores in an array

def assign_column_ner_classification(df,num_samples=25):
  df = df.sample(n=num_samples)
  col_types_df = pd.DataFrame(df)
  for colname in df.columns:
    col_types_df[colname] = df[colname].astype(str).apply(get_entities)

  return col_types_df

def major_col_pred_array(df,num_samples=25):
    majority_arr = []
    new_df = pd.DataFrame()
    new_df = assign_column_ner_classification(df,num_samples)
    majority_arr = np.empty(new_df.shape[1], dtype='object')


    for i, col in enumerate(new_df.columns):
    
      counts = new_df[col].value_counts()
      
      
      if counts.empty:
        majority_arr[i] = None
      else:
        
        majority_arr[i] = counts.index[0]
      
      
      if majority_arr[i]:
        print(f"{col}: {majority_arr[i]}")
      else:
        print(f"{col}: None")
    
    return majority_arr


In [ ]:
assign_column_ner_classification(df1,25)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
8371,[DATE],[PERSON],[PERSON],[CARDINAL],[DATE],[DATE],[],[CARDINAL],[CARDINAL],[CARDINAL],[DATE],[]
10404,[CARDINAL],"[NORP, PERSON, FAC, CARDINAL]","[PERSON, PERSON]",[CARDINAL],[DATE],[],[],[CARDINAL],[CARDINAL],[CARDINAL],[CARDINAL],[ORG]
1004,[DATE],[MONEY],[PERSON],[CARDINAL],[DATE],[DATE],[],[CARDINAL],[CARDINAL],[CARDINAL],[PERSON],[ORG]
6954,[DATE],[GPE],"[PERSON, PERSON]",[CARDINAL],[DATE],[DATE],[],[CARDINAL],[CARDINAL],[CARDINAL],[DATE],[]
4506,[DATE],[CARDINAL],[PERSON],[CARDINAL],[DATE],[DATE],[],[CARDINAL],[CARDINAL],[CARDINAL],[DATE],[]
6341,[DATE],[EVENT],[PERSON],[CARDINAL],[DATE],[DATE],[],[CARDINAL],[DATE],[CARDINAL],[CARDINAL],[]
10515,[DATE],[ORG],[PERSON],[CARDINAL],[DATE],[DATE],[GPE],[CARDINAL],[CARDINAL],[CARDINAL],[DATE],[]
6274,[DATE],"[PERSON, DATE, PERSON, MONEY]",[PERSON],[CARDINAL],[DATE],[DATE],[],[CARDINAL],[CARDINAL],[CARDINAL],[],[ORG]
10039,[DATE],[ORDINAL],[PERSON],[CARDINAL],[],[DATE],[],[CARDINAL],[DATE],[DATE],[DATE],[GPE]
5081,[DATE],"[GPE, DATE, FAC, PERSON]",[PERSON],[CARDINAL],[DATE],[DATE],[],[CARDINAL],[CARDINAL],[CARDINAL],[DATE],[ORG]


In [ ]:
# df storing the majority NER detected for each column, if not detected then column value = None
new_maj_col = major_col_pred_array(df1,25)

bookID: ['DATE']
title: None
authors: ['PERSON']
average_rating: ['CARDINAL']
isbn: ['DATE']
isbn13: ['DATE']
language_code: None
  num_pages: ['CARDINAL']
ratings_count: ['DATE']
text_reviews_count: ['CARDINAL']
publication_date: ['DATE']
publisher: ['PERSON']


In [ ]:
for i in new_maj_col:
  print(i)


new_maj_col.tolist()[2][0]

['DATE']
[]
['PERSON']
['CARDINAL']
['DATE']
['DATE']
[]
['CARDINAL']
['DATE']
['CARDINAL']
['DATE']
['PERSON']


'PERSON'

In [ ]:
columns = new_maj_col.tolist()
for i in range(12):
  columns[i]

In [ ]:
## need to make changes in this file for integration
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def match_column_type(dataset, column_name):
    # Create a TfidfVectorizer object
    vectorizer = TfidfVectorizer()
    
    # Fit the vectorizer on the text data
    vectorizer.fit(dataset[column_name])
    
    # Create a dictionary to store the similarity scores
    similarity_scores = {}
    
    # Iterate over the type map
    for key, values in type_map.items():
        # Create a string by joining the values
        values_str = ' '.join(values)
        
        # Transform the values string and the text data into vectors
        values_vec = vectorizer.transform([values_str])
        text_vec = vectorizer.transform(dataset[column_name])
        
        # Compute the cosine similarity between the values vector and the text vectors
        similarity = cosine_similarity(values_vec, text_vec)
        
        # Compute the average similarity score
        avg_similarity = similarity.mean()
        
        # Store the average similarity score in the dictionary
        similarity_scores[key] = avg_similarity
    
    # Find the key with the highest similarity score
    best_match = max(similarity_scores, key=similarity_scores.get)
    
    return best_match


import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import spacy

nlp = spacy.load("en_core_web_sm")

def predict_matching_values(df, column_name, key):
    
    num_columns = df.shape[1]
    num_columns=num_columns


    columns = new_maj_col.tolist()
    

    # Get the embeddings for the column name and key
    column_emb = nlp(str(column_name).lower()).vector.reshape(1, -1)
    key_emb = nlp(key).vector.reshape(1, -1)

    # Compute the cosine similarity between the column name and key embeddings
    sim = cosine_similarity(column_emb, key_emb)

    # Get the most similar value from the type map for the given key
    key_values = type_map[key]
    most_similar_value = max(key_values, key=lambda x: cosine_similarity(nlp(x).vector.reshape(1, -1), key_emb))

    # Create a list of values that closely match the column name
    matching_values = []
    for value in key_values:
        value_emb = nlp(value).vector.reshape(1, -1)
        value_sim = cosine_similarity(column_emb, value_emb)
        if value_sim >= sim:
          matching_values.append(value)

    return matching_values


def detect1_semantic_type(values):
    """Detects the semantic type of a list of numeric values."""
    
    # Initialize variables
    is_continuous = True
    is_discrete = True
    is_latitude = False
    is_longitude = False
    is_temperature = False
    is_idno = False
    is_altitude = False
    is_cost = False
    is_measurement = False
    is_salary = False
    
    # Calculate range and differences
    value_range = max(values) - min(values)
    value_diff = [values[i+1] - values[i] for i in range(len(values)-1)]
    diff_range = max(value_diff) - min(value_diff)
    # print(value_diff)
    # Check for discrete or continuous values
    if diff_range != 0:
        is_discrete = False
    else:
        is_continuous = False
    
    # # Check for latitude or longitude values
    if all(value >= -90 and value <= 90 and value > 10 for value in values):
        is_latitude = True
    elif all(value >= -180 and value <= 180 and value > 10 for value in values):
        is_longitude = True
        
    # Check for temperature values
    if value_range > 50 and all(value >= -10 and value <=50 for value in values):
        if all(abs(value) <= 10 for value in value_diff):
            is_temperature = True
        
    # Check for values
    if value_range <= 100 and all(value >= 0 and value <= 100 for value in values):
        is_idno = True
        
    # Check for altitude values
    if value_range > 1000 and all(value >= -413.6 and value <= 8848  for value in values):
        is_altitude = True
    
    # Check for cost values
    if value_range > 100 and any(value < 0 for value in values):
        is_cost = True
        
    # Check for measurement values
    if value_range > 1000 and any(value < 0 for value in values):
        is_measurement = True
        
    # Check for salary values
    if value_range > 1000 and all(value >= 0 for value in values):
        is_salary = True
        
    # Return results
    if is_latitude:
        return "Latitude"
    elif is_longitude:
        return "Longitude"
    elif is_temperature:
        return "Temperature"
    elif is_idno:
        return "ID number"
    elif is_altitude:
        return "Numeric Indicators"
    elif is_cost:
        return "Cost"
    elif is_measurement:
        return "Measurement"
    elif is_salary:
        return "Count"


def classify_string_column_three(col_text):
    # Define regex patterns for different entities
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    phone_pattern = r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b'
    pincode_pattern = r'\b\d{6}\b'
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    # Check if column matches any of the patterns
    if re.search(email_pattern, col_text):
        return 'email'
    elif re.search(phone_pattern, col_text):
        return 'phone number'
    elif re.search(pincode_pattern, col_text):
        return 'pincode'
    elif re.search(url_pattern, col_text):
        return 'url'
    else:
        return None


def classify_columns(data, batch_size=1000):
    data = data.iloc[:30,:]
    """
    Classifies columns in a CSV dataset using NER and regex-based entity classification.

    :param data: A Pandas DataFrame containing the CSV dataset.
    :param batch_size: The size of the batches to use for processing the dataset.
    :return: A Pandas DataFrame with column names and their predicted entity types.
    """
    predictions = []

    str_cols = [col for col in predicted_data_types if 'categorical' in predicted_data_types[col]]
    numeric_cols = [col for col in predicted_data_types if 'int' in predicted_data_types[col] or 'numerical' in predicted_data_types[col]]
    print()
    print("printing the columns : ")
    print("str: ",str_cols)
    print("numeric : ",numeric_cols)
    print()
    # Classify object type columns using NLP and regex

    i = 0
    for col in str_cols:
        flag_tm = 0
        # If no named entity is found, use regex to classify the column
        col_type = None
        col_text = ' '.join(data[col].fillna('').values)
        col_type = classify_string_column_three(col_text)


        if col_type:
          predictions.append({'colname': col, 'prediction' : col_type})


        if not col_type:
            num_columns = data.shape[1]
            columns = []
            columns = new_maj_col.tolist()
            
            i = data.columns.get_loc(col)
            if columns[i]:
              match_value = new_maj_col.tolist()[i][0]
              col_type =  match_value
              #col_type = predict_matching_values(data, col, columns[i], columns)
              col_vals = [word for word in type_map[match_value.lower()]]
              predictions.append({'colname': col, 'prediction': col_vals})
              #print("based on column ner type = ",col, col_type, col_vals)
              
            else:    
              col_text_word = col_text.split()[0]
              # print(col_text_word)
              if re.search(r"[MF]", col_text_word, re.IGNORECASE):
                  col_type = 'Gender'
                  predictions.append({'colname': col, 'prediction': col_type})
              elif re.search(r"^[a-zA-Z]$", col_text_word, re.IGNORECASE):
                  col_type = 'Class'
                  predictions.append({'colname': col, 'prediction': col_type})
              elif re.search(r"^\d$", col_text_word, re.IGNORECASE):
                  col_type = 'Number|Count'
                  predictions.append({'colname': col, 'prediction': col_type})
              else:
                  key = new_maj_col.tolist()[df1.columns.get_loc(col)]
                  if key:
                    col_type = predict_matching_values(data, col, key[0])
                    predictions.append({'colname': col, 'prediction': col_type})

                  else:
                    predictions.append({'colname': col, 'prediction': key})
                                     
    # Classify numerical columns using range-based hierarchical classification
    for col in numeric_cols:
        if detect1_semantic_type(data[col]):
          col_type = detect1_semantic_type(data[col])
          predictions.append({'colname': col, 'prediction': col_type})
                     
        else:
            col_range = data[col].max() - data[col].min()
          
            if col_range == 0:
                col_type = 'constant'
            elif col_range == 1:
                col_type = 'binary'
            elif col_range <= 10:
                col_type = 'ordinal'
            elif col_range <= 100:
                col_type = 'interval'
            else:
                col_type = 'ratio'
            predictions.append({'colname': col, 'prediction': col_type})
                               
    return pd.DataFrame(predictions)

In [ ]:
classify_columns(df1)


printing the columns : 
str:  ['title', 'authors', 'isbn', 'language_code', 'publication_date', 'publisher']
numeric :  ['bookID', 'isbn13', '  num_pages', 'ratings_count', 'text_reviews_count']



,colname,prediction
0,title,[]
1,authors,"[name, relation, first name, last name, middle..."
2,isbn,phone number
3,language_code,[]
4,publication_date,"[ID, serial number, identification number, bir..."
5,publisher,"[name, relation, first name, last name, middle..."
6,bookID,ID number
7,isbn13,Count
8,num_pages,Numeric Indicators
9,ratings_count,Count


In [ ]:
#for col in predicted_data_types:
#  print(new_maj_col.tolist()[df1.columns.get_loc(col)])